# Preprocess Required for Generating Train Data 

In [1]:
import urllib.request
import time
import sys
import getopt
import pandas as pd
import numpy as np
import pickle


In [2]:
%run ../utils.ipynb

In [3]:
embSize = 200
ftrain='../data/SNP_train_data.txt'
ftest = "../data/SNP_test_data.txt"
# Replace with path of word embdding file   
wefile = "/mnt/admin/GDA_backup/Dataset/embeddings/PubMed-and-PMC-w2v.bin"
random_seed=1331

# Read Train Data

In [4]:
samples= pd.read_csv(ftrain,sep='\t')
Tr_sent_contents, Tr_entity1_list, Tr_entity2_list, Tr_sent_lables,_ = dataRead_snp(ftrain)
Tr_word_list, Tr_d1_list, Tr_d2_list = get_wordList_and_distances_snp(Tr_sent_contents)
print ("train_size", len(Tr_word_list))

Input File Reading
train_size 935


# Read Test Data

In [5]:
Te_sent_contents, Te_entity1_list, Te_entity2_list, Te_sent_lables,_ = dataRead_snp(ftest)
Te_word_list, Te_d1_list, Te_d2_list = get_wordList_and_distances_snp(Te_sent_contents)

print ("test_size", len(Te_word_list))


Input File Reading
test_size 365


## Prepare Lable Matrix

In [6]:
set(Tr_sent_lables)

{'negative', 'neutral', 'positive'}

In [7]:
label_dict = {'negative':0, 'neutral':0,'positive':1}
Y_t = mapLabelToId_snp(Tr_sent_lables, label_dict)

Y_train = np.zeros((len(Y_t), 2))
for i in range(len(Y_t)):
    Y_train[i][Y_t[i]] = 1.0

In [8]:
Y_te = mapLabelToId_snp(Te_sent_lables, label_dict)

Y_test = np.zeros((len(Y_te), 2))
for i in range(len(Y_te)):
    Y_test[i][Y_te[i]] = 1.0

# Load integrated dataset to make a uniform word dictionary between source and target  Models

In [9]:
with open('data/integrated_entities.pickle', 'rb') as handle:    
    id_to_word = pickle.load(handle)
    word_to_id = pickle.load(handle)
    sentMax = pickle.load(handle)
    wv = pickle.load(handle)
    word_dict = pickle.load(handle)
    

    

# Generate Word and Position Embedding Vectors

### Word Embedding

In [10]:
#sent_list = sum([Tr_word_list, Te_word_list], [])
#word_dict, word_to_id, id_to_word = word_mapping(sent_list)
#print( "word dictonary length", len(word_dict))
# Word Embedding
word_vectors = readWordEmb(word_dict,id_to_word,word_to_id, wefile, embSize,limit=2000000)
W_train =   mapWordToId(Tr_word_list, word_to_id)

Reading word vectors
Loaded 2000000 pretrained embeddings.
number of unknown word in word embedding 3545


In [11]:
W_test =   mapWordToId(Te_word_list, word_to_id)

### Pad Embdding Vectors

In [12]:
#padding
W_train= paddData([W_train], sentMax) 
#padding
W_test = paddData([W_test], sentMax) 



# Save Training data as a Pickle file

In [13]:
with open('data/train_and_test_data_sentences_snp_2class_transfer.pickle', 'wb') as handle:
    pickle.dump(W_train, handle)
    pickle.dump(Y_train, handle)
    pickle.dump(Tr_word_list, handle)
    
    pickle.dump(W_test, handle)
    pickle.dump(Y_test, handle)
    pickle.dump(Te_word_list, handle)
    
    
    pickle.dump(label_dict, handle) 
    pickle.dump(sentMax, handle)
    
    

In [14]:
sentMax

100